In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.decomposition import PCA

# -- Utility functions --

def get_sorted_columns(A):
    return [np.sort(A[:, i]) for i in range(A.shape[1])]

def compute_1d_wasserstein_sorted(u_sorted, v_sorted):
    n, m = len(u_sorted), len(v_sorted)
    a = np.ones(n) / n
    b = np.ones(m) / m
    # For squared W2; change if you want W1
    import ot
    return ot.lp.emd2_1d(u_sorted, v_sorted, a, b)

def assignment_sliced_wasserstein(
    AX, BY, eigvals_X=None, eigvals_Y=None, eigvecs_X=None, eigvecs_Y=None, print_pairings=False
):
    k = AX.shape[1]
    AX_sorted = get_sorted_columns(AX)
    BY_sorted = get_sorted_columns(BY)
    C = np.zeros((k, k))
    signs = np.zeros((k, k))
    for i in range(k):
        a_sorted = AX_sorted[i]
        for j in range(k):
            b_sorted = BY_sorted[j]
            # Try positive
            d_pos = compute_1d_wasserstein_sorted(a_sorted, b_sorted)
            # Try negative (negate and reverse)
            b_neg_sorted = -b_sorted[::-1]
            d_neg = compute_1d_wasserstein_sorted(a_sorted, b_neg_sorted)
            if d_pos <= d_neg:
                C[i, j] = d_pos
                signs[i, j] = 1
            else:
                C[i, j] = d_neg
                signs[i, j] = -1
    row_ind, col_ind = linear_sum_assignment(C)
    matched_signs = np.array([signs[i, j] for i, j in zip(row_ind, col_ind)])
    if print_pairings and eigvals_X is not None and eigvecs_X is not None and eigvals_Y is not None and eigvecs_Y is not None:
        print("\nRISWIE Axis Pairings:")
        for idx in range(len(row_ind)):
            i, j = row_ind[idx], col_ind[idx]
            print(f"X axis {i}: eval={eigvals_X[i]:.3g}, evec={eigvecs_X[:,i]}   <--->   "
                  f"Y axis {j}: eval={eigvals_Y[j]:.3g}, evec={eigvecs_Y[:,j]}, "
                  f"contribution={C[i, j]:.6f}, sign={signs[i,j]:+}")
    return row_ind, col_ind, matched_signs, C

def pca_embedding_with_eigvals_and_evecs(X, k=None):
    Xc = X - X.mean(axis=0)
    k = k if k else X.shape[1]
    pca = PCA(n_components=k)
    pca.fit(Xc)
    AX = pca.transform(Xc)
    eigvals = pca.explained_variance_
    evecs = pca.components_.T  # d x k
    return AX, eigvals, evecs

import numpy as np
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh

def spectral_embedding_with_eigvals_and_evecs(
    X,
    k: int = None,
    n_neighbors: int = 10,
    normalized: bool = True
):
    """
    Spectral embedding (Laplacian Eigenmaps) -> coordinates + eigendata.

    Parameters
    ----------
    X : array (n_samples, n_features)
        Input data points.
    k : int, optional (default = None)
        Number of embedding dimensions.  If None, uses all nontrivial modes
        (i.e. n_samples-1).
    n_neighbors : int, optional (default=10)
        How many neighbors in the k-NN graph.
    normalized : bool, optional (default=True)
        Whether to use the normalized graph Laplacian.

    Returns
    -------
    embedding : array (n_samples, k)
        The spectral coordinates (each column is one eigenvector).
    eigvals : array (k,)
        The nontrivial eigenvalues of the Laplacian, in ascending order.
    eigvecs : array (n_samples, k)
        The corresponding eigenvectors (one per column).
    """
    n, _ = X.shape
    if k is None:
        k = n - 1

    # 1) build a symmetric k-NN graph
    A = kneighbors_graph(X, n_neighbors=n_neighbors,
                         include_self=True, metric='euclidean')
    A = 0.5 * (A + A.T)

    # 2) form the (normalized) Laplacian
    L = laplacian(A, normed=normalized)

    # 3) compute the smallest k+1 eigenpairs of L
    #    (the eigenvector for eigenvalue 0 is the trivial constant mode)
    vals, vecs = eigsh(L, k=k+1, which='SM')

    # 4) sort them (just in case)
    idx = np.argsort(vals)
    vals = vals[idx]
    vecs = vecs[:, idx]

    # 5) drop the first eigenpair (the trivial constant mode at ~0),
    #    keep the next k
    eigvals = vals[1 : k+1]
    eigvecs = vecs[:,   1 : k+1]

    # 6) your embedding is just the eigenvectors themselves
    embedding = eigvecs

    return embedding, eigvals, eigvecs


def align_by_riswie(X, Y, print_pairings=False):
    # X, Y: n x d
    AX, eigvals_X, eigvecs_X = pca_embedding_with_eigvals_and_evecs(X)
    BY, eigvals_Y, eigvecs_Y = pca_embedding_with_eigvals_and_evecs(Y)
    #AX, eigvals_X, eigvecs_X = spectral_embedding_with_eigvals_and_evecs(
    #X, k=300, n_neighbors=15, normalized=True
    #)
    #BY, eigvals_Y, eigvecs_Y = spectral_embedding_with_eigvals_and_evecs(
    #    Y, k=300, n_neighbors=15, normalized=True
    #)

    k = AX.shape[1]
    row_ind, col_ind, matched_signs, cost_matrix = assignment_sliced_wasserstein(
        AX, BY, eigvals_X, eigvals_Y, eigvecs_X, eigvecs_Y, print_pairings=print_pairings
    )

    # Construct signed permutation (from BY to AX)
    P = np.zeros((k, k))
    S = np.zeros((k, k))
    for i, (j, s) in enumerate(zip(col_ind, matched_signs)):
        P[i, j] = 1
        S[i, i] = s

    # Construct orthogonal transform: BY @ P.T @ S
    # This aligns BY axes (columns) to AX axes (possibly with sign flip)
    # For point clouds: transform BY so the j-th BY axis becomes the i-th AX axis (with sign)
    BY_aligned = BY @ P.T @ S
    return AX, BY_aligned, row_ind, col_ind, matched_signs

def get_word_pairs_by_hungarian(AX, BY_aligned, words_en, words_es):
    # AX, BY_aligned: n x d
    from scipy.spatial.distance import cdist
    C = cdist(AX, BY_aligned, metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(C)
    pred_pairs = [(words_en[i], words_es[j]) for i, j in zip(row_ind, col_ind)]
    return pred_pairs

def load_ground_truth_dict(filename):
    gt_dict = {}
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            en, es = line.strip().split()
            if en not in gt_dict:
                gt_dict[en] = set()
            gt_dict[en].add(es)
    return gt_dict



def compute_accuracy(pred_pairs, gt_dict):
    # only evaluate those in the dictionary
    valid = [(en,es) for en,es in pred_pairs if en in gt_dict]
    correct = sum(1 for en,es in valid if es in gt_dict[en])
    total   = len(valid)
    return correct/total, correct, total

from sklearn.metrics import pairwise_distances
import numpy as np

def recall_at_k(AX, BY_aligned, words_en, words_es, gt_dict, k=5):
    # Compute distance matrix (n_en, n_es)
    D = pairwise_distances(AX, BY_aligned, metric='euclidean')
    n = D.shape[0]
    correct = 0
    total = 0

    for i in range(n):
        en_word = words_en[i]
        if en_word not in gt_dict:
            continue  # skip if not in ground truth
        # Get top-k indices (smallest distances)
        topk_indices = np.argpartition(D[i], k)[:k]
        # What are their Spanish words?
        topk_es = {words_es[j] for j in topk_indices}
        # At least one match?
        if len(gt_dict[en_word] & topk_es) > 0:
            correct += 1
        total += 1
    recall = correct / total if total > 0 else 0.0
    print(f"Recall@{k}: {recall:.4f} ({correct} / {total})")
    return recall, correct, total

# -- Main workflow --
def main():
    # Load word embeddings
    with open('data/wiki.en.10k.vec', 'r', encoding='utf-8') as f_en, \
         open('data/wiki.es.10k.vec', 'r', encoding='utf-8') as f_es:
        words_en, X = read_vec(f_en)
        words_es, Y = read_vec(f_es)

    # Align by RISWIE
    AX, BY_aligned, row_ind, col_ind, matched_signs = align_by_riswie(X, Y, print_pairings=True)

    # Find pointwise matches (bijective) using Hungarian
    pred_pairs = get_word_pairs_by_hungarian(AX, BY_aligned, words_en, words_es)

    print("\nTop 100 Matched Pairs:")
    for idx, (en_word, es_word) in enumerate(pred_pairs[:100]):
        print(f"{idx+1:3d}: {en_word} ↔ {es_word}")

    # Score accuracy
    gt_dict = load_ground_truth_dict("data/en-es.0-6500.txt")
    accuracy, correct, total = compute_accuracy(pred_pairs, gt_dict)
    print(f"\nOverall Accuracy: {accuracy:.4f} ({correct} / {total} correct)")

    recall_at_k(AX, BY_aligned, words_en, words_es, gt_dict, k=5)

        
def read_vec(file, dtype='float'):
    header = file.readline().split(' ')
    count = int(header[0])
    dim = int(header[1])
    words = []
    matrix = np.empty((count, dim), dtype=dtype)
    for i in range(count):
        word, vec = file.readline().split(' ', 1)
        words.append(word)
        matrix[i] = np.fromstring(vec, sep=' ', dtype=dtype)
    print(f"Finished reading. Matrix shape: {matrix.shape}")
    return words, matrix

if __name__ == '__main__':
    main()


Finished reading. Matrix shape: (10000, 300)
Finished reading. Matrix shape: (10000, 300)

RISWIE Axis Pairings:
X axis 0: eval=1.05, evec=[ 0.03964034  0.02435579 -0.04506446 -0.03155119 -0.06529752  0.10500295
 -0.06712465 -0.02222605  0.09312945  0.04228634  0.04601559 -0.03207323
  0.10875884  0.13379581 -0.00138458  0.03274927 -0.01722203  0.00700942
  0.04127298 -0.02539479 -0.0326851   0.00799478  0.02811715  0.00264889
  0.18929455  0.03829655 -0.01937214  0.02138493 -0.04163972 -0.10355845
  0.02869261 -0.06014388  0.02270803 -0.08303041 -0.04814231  0.03028888
 -0.0082872  -0.03830875  0.05146717  0.04945185  0.01580282  0.06907261
 -0.00565507  0.02508217 -0.08642212 -0.01738128 -0.04778509  0.00686902
 -0.02979528 -0.0186269   0.03089284  0.15171357  0.0255881   0.01926072
  0.03692101  0.02925596 -0.05602205  0.01845129  0.06538593 -0.01475781
  0.11977483 -0.07639846 -0.02978452 -0.09154962 -0.00961215 -0.01147578
 -0.03639875 -0.07524086  0.07654504  0.01596854 -0.041627

In [2]:
# Quick check: How many unique English words in gold dict?
gt_dict = load_ground_truth_dict('data/en-es.0-6500.txt')
print("Number of English words in gold:", len(gt_dict))


Number of English words in gold: 6500


In [8]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cdist
import ot
from sklearn.decomposition import PCA

# ——— I/O ———————————————————————————————————————————————————————
def read_vec(f, dtype='float'):
    header = f.readline().split()
    n, d = map(int, header)
    words = []
    M = np.empty((n, d), dtype=dtype)
    for i in range(n):
        w, vec = f.readline().split(' ', 1)
        words.append(w)
        M[i] = np.fromstring(vec, sep=' ', dtype=dtype)
    print(f"Loaded {n}×{d}")
    return words, M

def load_ground_truth_dict(path):
    gt = {}
    for L in open(path, encoding='utf-8'):
        e, s = L.strip().split()
        gt.setdefault(e, set()).add(s)
    return gt

def score(pred, gt):
    c = sum(1 for e,s in pred if e in gt and s in gt[e])
    return c/len(pred), c, len(pred)

# ——— 1D-Wass utility —————————————————————————————————————————————
def get_sorted_columns(A):
    return [np.sort(A[:,i]) for i in range(A.shape[1])]

def w2_sq_1d(u_sorted, v_sorted):
    a = np.ones(len(u_sorted))/len(u_sorted)
    b = np.ones(len(v_sorted))/len(v_sorted)
    return ot.lp.emd2_1d(u_sorted, v_sorted, a, b)

# ——— Spectral embedding (or swap PCA) ————————————————————————————
def spectral_embed(X, k, n_neighbors=10, normalized=True):
    n, _ = X.shape
    G = kneighbors_graph(X, n_neighbors, include_self=True).astype(float)
    G = 0.5*(G+G.T)
    L = laplacian(G, normed=normalized)
    vals, vecs = eigsh(L, k=k+1, which='SM')
    idx = np.argsort(vals)
    vals, vecs = vals[idx], vecs[:,idx]
    return vecs[:,1:k+1], vals[1:k+1], vecs[:,1:k+1]

# ——— RISWIE axis match & cost —————————————————————————————————————
def riswie_match_cost(AX, BY):
    # AX, BY: (n, k)
    k = AX.shape[1]
    Acols = get_sorted_columns(AX)
    Bcols = get_sorted_columns(BY)
    C = np.zeros((k,k))
    for i in range(k):
        a = Acols[i]
        for j in range(k):
            b = Bcols[j]
            dpos = w2_sq_1d(a, b)
            dneg = w2_sq_1d(a, -b[::-1])
            C[i,j] = min(dpos, dneg)
    ri, ci = linear_sum_assignment(C)
    cost = C[ri,ci].sum()/k
    return cost, ri, ci

# ——— Ortho-align subclouds via RISWIE axes ——————————————————————————
def subcloud_align(Xc, Yc, k):
    """
    Given two centered subclouds Xc, Yc in R^d,
    (n_i x d) each, do PCA->RISWIE to get T in R^{dxd}.
    """
    n_i, d = Xc.shape

    # 1) run PCA in feature‐space so Ux,Uy ∈ R^{d × k}
    pcaX = PCA(n_components=k).fit(Xc)
    Ux = pcaX.components_.T            # shape (d, k)
    pcaY = PCA(n_components=k).fit(Yc)
    Uy = pcaY.components_.T            # shape (d, k)

    # 2) project to get 1D coords
    AX = Xc @ Ux                       # (n_i, k)
    BY = Yc @ Uy                       # (n_i, k)

    # 3) match axes via RISWIE
    _, ri, ci = riswie_match_cost(AX, BY)

    # 4) build signed permutation P,S in R^{k×k}
    P = np.zeros((k, k))
    S = np.zeros((k, k))
    for i, j in zip(ri, ci):
        P[i, j] = 1
        # pick sign by comparing the two 1D W2 costs
        a = np.sort(AX[:, i])
        b = np.sort(BY[:, j])
        if w2_sq_1d(a, b) <= w2_sq_1d(a, -b[::-1]):
            sgn = 1
        else:
            sgn = -1
        S[i, i] = sgn

    # 5) build the d×d orthogonal T
    T = Ux.dot(P.T).dot(S).dot(Uy.T)
    return T
# ——— Main pipeline —————————————————————————————————————————————
def main(
    en_path='data/wiki.en.10k.vec',
    es_path='data/wiki.es.10k.vec',
    gt_path='data/en-es.0-6500.txt',
    n_clusters=16,
    embed_dim=1000
):
    # 1) load
    words_en, X = read_vec(open(en_path,'r',encoding='utf-8'))
    words_es, Y = read_vec(open(es_path,'r',encoding='utf-8'))
    n = len(X)

    # 2) cluster each
    kmX = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    kmY = KMeans(n_clusters=n_clusters, random_state=1).fit(Y)
    assignX = kmX.labels_
    assignY = kmY.labels_

    # 3) cost between every cluster pair
    Cc = np.zeros((n_clusters,n_clusters))
    for i in range(n_clusters):
        Xi = X[assignX==i]
        Xi = Xi - Xi.mean(0)
        for j in range(n_clusters):
            Yj = Y[assignY==j]
            Yj = Yj - Yj.mean(0)
            if len(Xi)>embed_dim and len(Yj)>embed_dim:
                cost,_,_ = riswie_match_cost(
                    spectral_embed(Xi,embed_dim)[0],
                    spectral_embed(Yj,embed_dim)[0]
                )
            else:
                cost = np.inf
            Cc[i,j]=cost

    # 4) match clusters
    rc, cc = linear_sum_assignment(Cc)

    # 5) within each matched pair do fine align+Hungarian
    pred = []
    for i,j in zip(rc,cc):
        Xi = X[assignX==i]; Yi = Y[assignY==j]
        ci = Xi.mean(0); cj = Yi.mean(0)
        Ti = subcloud_align(Xi-ci, Yi-cj, embed_dim)
        # transform Y→X
        Yi_al = (Yi-cj).dot(Ti.T) + ci
        # final Hungarian on points
        D = cdist(Xi, Yi_al)
        ri, ci2 = linear_sum_assignment(D)
        for a,b in zip(ri,ci2):
            pred.append(( words_en[np.where((X==Xi[a]).all(1))[0][0]],
                          words_es[np.where((Y==Yi[b]).all(1))[0][0]] ))

    # 6) evaluate
    gt = load_ground_truth_dict(gt_path)
    acc, corr, tot = score(pred, gt)

    print(f"Matched {len(pred)} words  →  Acc={acc:.4f} ({corr}/{tot})")
    for idx,(e,s) in enumerate(pred[:100],1):
        print(f"{idx:3d}: {e} ↔ {s}")

if __name__=='__main__':
    main()


Loaded 10000×300
Loaded 10000×300


ValueError: cost matrix is infeasible

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Graph–embedding robustness experiment with RISWIE distance
===========================================================

Ten datasets := 5 clusters of English words  +  5 clusters of Spanish words
For each dataset build 10 graphs (4 k-NN, 3 r-NN, 3 ε graphs)
Compute spectral embeddings (d = 30)                   -> 100 embeddings
Compute 100×100 RISWIE distance matrix
For every graph pick its 9 nearest neighbours (exclude self) and
count how many belong to the same dataset (cluster) but use a different graph.
Return mean accuracy over 100 graphs.
"""

import os
import itertools
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix, csgraph, diags
from scipy.sparse.linalg import eigsh
from scipy.optimize import linear_sum_assignment

from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import ot  # POT library


# ---------------------------------------------------------------------
# I/O helpers ----------------------------------------------------------

def read_vec(path, dtype="float32"):
    """Read a .vec or .txt fastText file"""
    with open(path, "r", encoding="utf-8") as f:
        hdr = f.readline().split()
        n, dim = int(hdr[0]), int(hdr[1])
        words, mat = [], np.empty((n, dim), dtype=dtype)
        for i in range(n):
            w, vec = f.readline().split(" ", 1)
            words.append(w)
            mat[i] = np.fromstring(vec, sep=" ", dtype=dtype)
    return words, mat


# ---------------------------------------------------------------------
# Graph construction ---------------------------------------------------

def knn_graph(X, k):
    A = kneighbors_graph(X, n_neighbors=k, include_self=False, metric="euclidean")
    A = 0.5 * (A + A.T)           # symmetrise
    return A.tocsr()

def radius_graph(X, radius):
    D = squareform(pdist(X, metric="euclidean"))
    A = (D <= radius).astype(float)
    np.fill_diagonal(A, 0.)
    return csr_matrix(A)

def epsilon_graph(X, epsilon):
    D = squareform(pdist(X, metric="euclidean"))
    W = np.exp(-(D**2) / (2*epsilon**2))
    W[D > 3*epsilon] = 0.                # sparsify far edges
    np.fill_diagonal(W, 0.)
    return csr_matrix(W)


# ---------------------------------------------------------------------
# Spectral embedding (Laplacian Eigenmaps) -----------------------------

def spectral_embed_graph(A, dim=30, normalised=True):
    """
    Return (n × dim) matrix of the first *dim* non-trivial eigenvectors.
    """
    # Compute Laplacian
    L = csgraph.laplacian(A, normed=normalised)
    n = L.shape[0]
    # Smallest (dim+1) eigenpairs (skip λ≈0 constant mode)
    vals, vecs = eigsh(L, k=dim+1, which="SM")
    idx = np.argsort(vals)        # ascending
    vals, vecs = vals[idx][1:], vecs[:, idx][:, 1:]
    return vecs, vals             # embedding, eigenvalues


# ---------------------------------------------------------------------
# RISWIE utilities -----------------------------------------------------

def _sorted_cols(M):
    return [np.sort(M[:, j]) for j in range(M.shape[1])]

def _w2_1d_sorted(u, v):
    n, m = len(u), len(v)
    a, b = np.ones(n)/n, np.ones(m)/m
    return ot.lp.emd2_1d(u, v, a, b)     # Wasserstein-2²

def riswie_distance(A, B):
    """
    Rigid-Invariant Sliced Wasserstein distance between two embeddings A,B
      A, B: (n × d) with same n and d
    Returns: scalar distance
    """
    d = A.shape[1]
    A_sorted, B_sorted = _sorted_cols(A), _sorted_cols(B)

    # Cost matrix over axes
    C = np.zeros((d, d))
    signs = np.ones((d, d))
    for i in range(d):
        u = A_sorted[i]
        for j in range(d):
            v = B_sorted[j]
            pos = _w2_1d_sorted(u, v)
            neg = _w2_1d_sorted(u, -v[::-1])
            if pos <= neg:
                C[i, j] = pos
                signs[i, j] = 1
            else:
                C[i, j] = neg
                signs[i, j] = -1

    ri, ci = linear_sum_assignment(C)
    cost = C[ri, ci].mean()               # mean squared Wasserstein
    return np.sqrt(cost)                  # RISWIE “distance”


# ---------------------------------------------------------------------
# Pipeline -------------------------------------------------------------

def build_all_graphs(cloud, recipe_id_list):
    """
    cloud: (n,d) numpy array
    recipe_id_list: list of 10 graph construction specs
    returns: list[csr_matrix]
    """
    graphs = []
    Dmat = squareform(pdist(cloud))
    med = np.median(Dmat[Dmat > 0])
    # compute radii percentiles
    dist_vals = Dmat[np.triu_indices_from(Dmat, k=1)]
    p5, p10, p20 = np.percentile(dist_vals, [5, 10, 20])

    for recipe in recipe_id_list:
        typ, param = recipe
        if typ == "knn":
            graphs.append(knn_graph(cloud, k=param))
        elif typ == "r":
            graphs.append(radius_graph(cloud, radius=param))
        elif typ == "eps":
            graphs.append(epsilon_graph(cloud, epsilon=param))
        else:
            raise ValueError(f"unknown recipe {recipe}")
    return graphs


def main():
    # --------------------------------------------------------------
    # 1) load embeddings (top-10k words each)
    en_words, X_en  = read_vec("data/wiki.en.10k.vec")
    es_words, X_es  = read_vec("data/wiki.es.10k.vec")

    # --------------------------------------------------------------
    # 2) k-means-5 clusters per language
    kmeans_en = KMeans(n_clusters=5, random_state=42).fit(X_en)
    kmeans_es = KMeans(n_clusters=5, random_state=42).fit(X_es)

    clusters = []        # list of (lang_tag, cluster_id, pointcloud)
    for cid in range(5):
        clusters.append(("EN", cid, X_en[kmeans_en.labels_ == cid]))
        clusters.append(("ES", cid, X_es[kmeans_es.labels_ == cid]))

    # --------------------------------------------------------------
    # 3) recipes: 10 graph constructions
    graph_recipes = [
        ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
        ("r",   None), ("r", None), ("r", None),          # placeholders – will be filled later
        ("eps", None), ("eps", None), ("eps", None)       # three ε graphs
    ]
    # we'll fill r- and eps- parameters inside build_all_graphs

    # --------------------------------------------------------------
    # 4) build graphs & spectral embeddings
    embeddings = []          # list of (dataset_id, graph_id, embedding (n×d))
    dataset_tags = []        # length-100 label (lang,cluster)
    global_idx = 0

    for ds_id, (lang, cid, cloud) in enumerate(clusters):
        # compute once the distance statistics for radius / ε graphs
        Dvals = pdist(cloud)
        p5, p10, p20 = np.percentile(Dvals, [5, 10, 20])
        med = np.median(Dvals)

        # clone recipes with concrete params
        concrete_recipes = [
            ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
            ("r", p5), ("r", p10), ("r", p20),
            ("eps", 0.6*med), ("eps", 0.8*med), ("eps", 1.0*med)
        ]

        graphs = build_all_graphs(cloud, concrete_recipes)

        for g_id, A in enumerate(graphs):
            emb, _ = spectral_embed_graph(A, dim=30, normalised=True)
            embeddings.append((ds_id, g_id, emb))
            dataset_tags.append(ds_id)
            global_idx += 1

    assert len(embeddings) == 100

    # --------------------------------------------------------------
    # 5) pairwise RISWIE distances
    n_graphs = len(embeddings)
    dist_mat = np.zeros((n_graphs, n_graphs))

    print("Computing 100 × 100 RISWIE distances …")
    for i, (_, _, Ei) in enumerate(tqdm(embeddings)):
        for j in range(i+1, n_graphs):
            Ej = embeddings[j][2]
            d = riswie_distance(Ei, Ej)
            dist_mat[i, j] = dist_mat[j, i] = d

    # --------------------------------------------------------------
    # 6) nearest-neighbour consistency score
    correct = 0
    for i in range(n_graphs):
        # indices sorted by distance (ascending), exclude self
        neighbours = np.argsort(dist_mat[i])[1:10]       # 9 nearest
        # how many share the same dataset id?
        same = sum(dataset_tags[j] == dataset_tags[i] for j in neighbours)
        correct += same

    accuracy = correct / (n_graphs * 9)
    print(f"\nNearest-neighbour consistency accuracy: {accuracy:.4f}")


# ---------------------------------------------------------------------
if __name__ == "__main__":
    main()


Computing 100 × 100 RISWIE distances …


100%|██████████| 100/100 [22:40<00:00, 13.60s/it]


Nearest-neighbour consistency accuracy: 0.3333


In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Graph–embedding robustness experiment with RISWIE distance
===========================================================

Ten datasets := 5 clusters of English words  +  5 clusters of Spanish words
For each dataset build 10 graphs (4 k-NN, 3 r-NN, 3 ε graphs)
Compute spectral embeddings (d = 30)                   -> 100 embeddings
Compute 100×100 RISWIE distance matrix
For every graph pick its 9 nearest neighbours (exclude self) and
count how many belong to the same dataset (cluster) but use a different graph.
Return mean accuracy over 100 graphs.
"""

import os
import itertools
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix, csgraph, diags
from scipy.sparse.linalg import eigsh
from scipy.optimize import linear_sum_assignment

from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import ot  # POT library


# ---------------------------------------------------------------------
# I/O helpers ----------------------------------------------------------

def read_vec(path, dtype="float32"):
    """Read a .vec or .txt fastText file"""
    with open(path, "r", encoding="utf-8") as f:
        hdr = f.readline().split()
        n, dim = int(hdr[0]), int(hdr[1])
        words, mat = [], np.empty((n, dim), dtype=dtype)
        for i in range(n):
            w, vec = f.readline().split(" ", 1)
            words.append(w)
            mat[i] = np.fromstring(vec, sep=" ", dtype=dtype)
    return words, mat


# ---------------------------------------------------------------------
# Graph construction ---------------------------------------------------

def knn_graph(X, k):
    A = kneighbors_graph(X, n_neighbors=k, include_self=False, metric="euclidean")
    A = 0.5 * (A + A.T)           # symmetrise
    return A.tocsr()

def radius_graph(X, radius):
    D = squareform(pdist(X, metric="euclidean"))
    A = (D <= radius).astype(float)
    np.fill_diagonal(A, 0.)
    return csr_matrix(A)

def epsilon_graph(X, epsilon):
    D = squareform(pdist(X, metric="euclidean"))
    W = np.exp(-(D**2) / (2*epsilon**2))
    W[D > 3*epsilon] = 0.                # sparsify far edges
    np.fill_diagonal(W, 0.)
    return csr_matrix(W)


# ---------------------------------------------------------------------
# Spectral embedding (Laplacian Eigenmaps) -----------------------------

def spectral_embed_graph(A, dim=30, normalised=True):
    """
    Return (n × dim) matrix of the first *dim* non-trivial eigenvectors.
    """
    # Compute Laplacian
    L = csgraph.laplacian(A, normed=normalised)
    n = L.shape[0]
    # Smallest (dim+1) eigenpairs (skip λ≈0 constant mode)
    vals, vecs = eigsh(L, k=dim+1, which="SM")
    idx = np.argsort(vals)        # ascending
    vals, vecs = vals[idx][1:], vecs[:, idx][:, 1:]
    return vecs, vals             # embedding, eigenvalues


# ---------------------------------------------------------------------
# RISWIE utilities -----------------------------------------------------

def _sorted_cols(M):
    return [np.sort(M[:, j]) for j in range(M.shape[1])]

def _w2_1d_sorted(u, v):
    n, m = len(u), len(v)
    a, b = np.ones(n)/n, np.ones(m)/m
    return ot.lp.emd2_1d(u, v, a, b)     # Wasserstein-2²

def riswie_distance(A, B):
    """
    Rigid-Invariant Sliced Wasserstein distance between two embeddings A,B
      A, B: (n × d) with same n and d
    Returns: scalar distance
    """
    d = A.shape[1]
    A_sorted, B_sorted = _sorted_cols(A), _sorted_cols(B)

    # Cost matrix over axes
    C = np.zeros((d, d))
    signs = np.ones((d, d))
    for i in range(d):
        u = A_sorted[i]
        for j in range(d):
            v = B_sorted[j]
            pos = _w2_1d_sorted(u, v)
            neg = _w2_1d_sorted(u, -v[::-1])
            if pos <= neg:
                C[i, j] = pos
                signs[i, j] = 1
            else:
                C[i, j] = neg
                signs[i, j] = -1

    ri, ci = linear_sum_assignment(C)
    cost = C[ri, ci].mean()               # mean squared Wasserstein
    return np.sqrt(cost)                  # RISWIE “distance”

def sliced_wasserstein_distance(A, B, n_proj=300, seed=0):
    """
    Computes Sliced Wasserstein-2 distance between two point clouds (can have different numbers of points).
    A: (n, d) numpy array
    B: (m, d) numpy array
    n_proj: number of random projection directions
    Returns: scalar SW2 distance
    """
    n, d = A.shape
    m, dB = B.shape
    assert d == dB
    rng = np.random.default_rng(seed)
    sw_vals = []
    for _ in range(n_proj):
        # Random unit vector in R^d
        v = rng.normal(size=(d,))
        v /= np.linalg.norm(v)
        # Project points
        proj_A = np.dot(A, v)
        proj_B = np.dot(B, v)
        # Sort projections
        proj_A.sort()
        proj_B.sort()
        # Wasserstein-2 (squared), allowing different sizes
        a = np.ones(n) / n
        b = np.ones(m) / m
        w2 = ot.lp.emd2_1d(proj_A, proj_B, a, b)
        sw_vals.append(w2)
    return np.sqrt(np.mean(sw_vals))   # Return SW2 (not squared)

# ---------------------------------------------------------------------
# Pipeline -------------------------------------------------------------

def build_all_graphs(cloud, recipe_id_list):
    """
    cloud: (n,d) numpy array
    recipe_id_list: list of 10 graph construction specs
    returns: list[csr_matrix]
    """
    graphs = []
    Dmat = squareform(pdist(cloud))
    med = np.median(Dmat[Dmat > 0])
    # compute radii percentiles
    dist_vals = Dmat[np.triu_indices_from(Dmat, k=1)]
    p5, p10, p20 = np.percentile(dist_vals, [5, 10, 20])

    for recipe in recipe_id_list:
        typ, param = recipe
        if typ == "knn":
            graphs.append(knn_graph(cloud, k=param))
        elif typ == "r":
            graphs.append(radius_graph(cloud, radius=param))
        elif typ == "eps":
            graphs.append(epsilon_graph(cloud, epsilon=param))
        else:
            raise ValueError(f"unknown recipe {recipe}")
    return graphs


def main():
    # --------------------------------------------------------------
    # 1) load embeddings (top-10k words each)
    en_words, X_en  = read_vec("data/wiki.en.10k.vec")
    es_words, X_es  = read_vec("data/wiki.es.10k.vec")

    # --------------------------------------------------------------
    # 2) k-means-5 clusters per language
    kmeans_en = KMeans(n_clusters=5, random_state=42).fit(X_en)
    kmeans_es = KMeans(n_clusters=5, random_state=42).fit(X_es)

    clusters = []        # list of (lang_tag, cluster_id, pointcloud)
    for cid in range(5):
        clusters.append(("EN", cid, X_en[kmeans_en.labels_ == cid]))
        clusters.append(("ES", cid, X_es[kmeans_es.labels_ == cid]))

    # --------------------------------------------------------------
    # 3) recipes: 10 graph constructions
    graph_recipes = [
        ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
        ("r",   None), ("r", None), ("r", None),          # placeholders – will be filled later
        ("eps", None), ("eps", None), ("eps", None)       # three ε graphs
    ]
    # we'll fill r- and eps- parameters inside build_all_graphs

    # --------------------------------------------------------------
    # 4) build graphs & spectral embeddings
    embeddings = []          # list of (dataset_id, graph_id, embedding (n×d))
    dataset_tags = []        # length-100 label (lang,cluster)
    global_idx = 0

    for ds_id, (lang, cid, cloud) in enumerate(clusters):
        # compute once the distance statistics for radius / ε graphs
        Dvals = pdist(cloud)
        p5, p10, p20 = np.percentile(Dvals, [5, 10, 20])
        med = np.median(Dvals)

        # clone recipes with concrete params
        concrete_recipes = [
            ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
            ("r", p5), ("r", p10), ("r", p20),
            ("eps", 0.6*med), ("eps", 0.8*med), ("eps", 1.0*med)
        ]

        graphs = build_all_graphs(cloud, concrete_recipes)

        for g_id, A in enumerate(graphs):
            emb, _ = spectral_embed_graph(A, dim=30, normalised=True)
            embeddings.append((ds_id, g_id, emb))
            dataset_tags.append(ds_id)
            global_idx += 1

    assert len(embeddings) == 100

    # --------------------------------------------------------------
    # 5) pairwise RISWIE distances
    n_graphs = len(embeddings)
    dist_mat = np.zeros((n_graphs, n_graphs))

    print("Computing 100 × 100 RISWIE distances …")
    for i, (_, _, Ei) in enumerate(tqdm(embeddings)):
        for j in range(i+1, n_graphs):
            Ej = embeddings[j][2]
            #d = riswie_distance(Ei, Ej)
            d = sliced_wasserstein_distance(Ei, Ej)
            dist_mat[i, j] = dist_mat[j, i] = d

    # --------------------------------------------------------------
    # 6) nearest-neighbour consistency score
    correct = 0
    for i in range(n_graphs):
        # indices sorted by distance (ascending), exclude self
        neighbours = np.argsort(dist_mat[i])[1:10]       # 9 nearest
        # how many share the same dataset id?
        same = sum(dataset_tags[j] == dataset_tags[i] for j in neighbours)
        correct += same

    accuracy = correct / (n_graphs * 9)
    print(f"\nNearest-neighbour consistency accuracy: {accuracy:.4f}")


# ---------------------------------------------------------------------
if __name__ == "__main__":
    main()


Computing 100 × 100 RISWIE distances …


100%|██████████| 100/100 [17:49<00:00, 10.70s/it]


Nearest-neighbour consistency accuracy: 0.3900


In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Graph–embedding robustness experiment with RISWIE distance
===========================================================

Ten datasets := 5 clusters of English words  +  5 clusters of Spanish words
For each dataset build 10 graphs (4 k-NN, 3 r-NN, 3 ε graphs)
Compute spectral embeddings (d = 30)                   -> 100 embeddings
Compute 100×100 RISWIE distance matrix
For every graph pick its 9 nearest neighbours (exclude self) and
count how many belong to the same dataset (cluster) but use a different graph.
Return mean accuracy over 100 graphs.
"""

import os
import itertools
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix, csgraph, diags
from scipy.sparse.linalg import eigsh
from scipy.optimize import linear_sum_assignment

from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import ot  # POT library


# ---------------------------------------------------------------------
# I/O helpers ----------------------------------------------------------

def read_vec(path, dtype="float32"):
    """Read a .vec or .txt fastText file"""
    with open(path, "r", encoding="utf-8") as f:
        hdr = f.readline().split()
        n, dim = int(hdr[0]), int(hdr[1])
        words, mat = [], np.empty((n, dim), dtype=dtype)
        for i in range(n):
            w, vec = f.readline().split(" ", 1)
            words.append(w)
            mat[i] = np.fromstring(vec, sep=" ", dtype=dtype)
    return words, mat


# ---------------------------------------------------------------------
# Graph construction ---------------------------------------------------

def knn_graph(X, k):
    A = kneighbors_graph(X, n_neighbors=k, include_self=False, metric="euclidean")
    A = 0.5 * (A + A.T)           # symmetrise
    return A.tocsr()

def radius_graph(X, radius):
    D = squareform(pdist(X, metric="euclidean"))
    A = (D <= radius).astype(float)
    np.fill_diagonal(A, 0.)
    return csr_matrix(A)

def epsilon_graph(X, epsilon):
    D = squareform(pdist(X, metric="euclidean"))
    W = np.exp(-(D**2) / (2*epsilon**2))
    W[D > 3*epsilon] = 0.                # sparsify far edges
    np.fill_diagonal(W, 0.)
    return csr_matrix(W)


# ---------------------------------------------------------------------
# Spectral embedding (Laplacian Eigenmaps) -----------------------------

def spectral_embed_graph(A, dim=30, normalised=True):
    """
    Return (n × dim) matrix of the first *dim* non-trivial eigenvectors.
    """
    # Compute Laplacian
    L = csgraph.laplacian(A, normed=normalised)
    n = L.shape[0]
    # Smallest (dim+1) eigenpairs (skip λ≈0 constant mode)
    vals, vecs = eigsh(L, k=dim+1, which="SM")
    idx = np.argsort(vals)        # ascending
    vals, vecs = vals[idx][1:], vecs[:, idx][:, 1:]
    return vecs, vals             # embedding, eigenvalues


# ---------------------------------------------------------------------
# RISWIE utilities -----------------------------------------------------

def _sorted_cols(M):
    return [np.sort(M[:, j]) for j in range(M.shape[1])]

def _w2_1d_sorted(u, v):
    n, m = len(u), len(v)
    a, b = np.ones(n)/n, np.ones(m)/m
    return ot.lp.emd2_1d(u, v, a, b)     # Wasserstein-2²

def riswie_distance(A, B):
    """
    Rigid-Invariant Sliced Wasserstein distance between two embeddings A,B
      A, B: (n × d) with same n and d
    Returns: scalar distance
    """
    d = A.shape[1]
    A_sorted, B_sorted = _sorted_cols(A), _sorted_cols(B)

    # Cost matrix over axes
    C = np.zeros((d, d))
    signs = np.ones((d, d))
    for i in range(d):
        u = A_sorted[i]
        for j in range(d):
            v = B_sorted[j]
            pos = _w2_1d_sorted(u, v)
            neg = _w2_1d_sorted(u, -v[::-1])
            if pos <= neg:
                C[i, j] = pos
                signs[i, j] = 1
            else:
                C[i, j] = neg
                signs[i, j] = -1

    ri, ci = linear_sum_assignment(C)
    cost = C[ri, ci].mean()               # mean squared Wasserstein
    return np.sqrt(cost)                  # RISWIE “distance”


def gromov_wasserstein(X, Y):
    C1 = cdist(X, X)  # pairwise distance matrix for X
    C2 = cdist(Y, Y)  # pairwise distance matrix for Y
    # we assume uniform distributions for both sets
    p = np.ones(len(X)) / len(X)
    q = np.ones(len(Y)) / len(Y)
    # GW returns the cost directly (not the coupling)
    gw_cost = ot.gromov.gromov_wasserstein2(C1, C2, p, q, 'square_loss')
    return np.sqrt(gw_cost) # we need to square root because assignment_sliced_wasserstein returns sqrt of cost and we want to be consistent


# ---------------------------------------------------------------------
# Pipeline -------------------------------------------------------------

def build_all_graphs(cloud, recipe_id_list):
    """
    cloud: (n,d) numpy array
    recipe_id_list: list of 10 graph construction specs
    returns: list[csr_matrix]
    """
    graphs = []
    Dmat = squareform(pdist(cloud))
    med = np.median(Dmat[Dmat > 0])
    # compute radii percentiles
    dist_vals = Dmat[np.triu_indices_from(Dmat, k=1)]
    p5, p10, p20 = np.percentile(dist_vals, [5, 10, 20])

    for recipe in recipe_id_list:
        typ, param = recipe
        if typ == "knn":
            graphs.append(knn_graph(cloud, k=param))
        elif typ == "r":
            graphs.append(radius_graph(cloud, radius=param))
        elif typ == "eps":
            graphs.append(epsilon_graph(cloud, epsilon=param))
        else:
            raise ValueError(f"unknown recipe {recipe}")
    return graphs


def main():
    # --------------------------------------------------------------
    # 1) load embeddings (top-10k words each)
    en_words, X_en  = read_vec("data/wiki.en.10k.vec")
    es_words, X_es  = read_vec("data/wiki.es.10k.vec")

    # --------------------------------------------------------------
    # 2) k-means-5 clusters per language
    kmeans_en = KMeans(n_clusters=5, random_state=42).fit(X_en)
    kmeans_es = KMeans(n_clusters=5, random_state=42).fit(X_es)

    clusters = []        # list of (lang_tag, cluster_id, pointcloud)
    for cid in range(5):
        clusters.append(("EN", cid, X_en[kmeans_en.labels_ == cid]))
        clusters.append(("ES", cid, X_es[kmeans_es.labels_ == cid]))

    # --------------------------------------------------------------
    # 3) recipes: 10 graph constructions
    graph_recipes = [
        ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
        ("r",   None), ("r", None), ("r", None),          # placeholders – will be filled later
        ("eps", None), ("eps", None), ("eps", None)       # three ε graphs
    ]
    # we'll fill r- and eps- parameters inside build_all_graphs

    # --------------------------------------------------------------
    # 4) build graphs & spectral embeddings
    embeddings = []          # list of (dataset_id, graph_id, embedding (n×d))
    dataset_tags = []        # length-100 label (lang,cluster)
    global_idx = 0

    for ds_id, (lang, cid, cloud) in enumerate(clusters):
        # compute once the distance statistics for radius / ε graphs
        Dvals = pdist(cloud)
        p5, p10, p20 = np.percentile(Dvals, [5, 10, 20])
        med = np.median(Dvals)

        # clone recipes with concrete params
        concrete_recipes = [
            ("knn", 5), ("knn", 10), ("knn", 15), ("knn", 30),
            ("r", p5), ("r", p10), ("r", p20),
            ("eps", 0.6*med), ("eps", 0.8*med), ("eps", 1.0*med)
        ]

        graphs = build_all_graphs(cloud, concrete_recipes)

        for g_id, A in enumerate(graphs):
            emb, _ = spectral_embed_graph(A, dim=30, normalised=True)
            embeddings.append((ds_id, g_id, emb))
            dataset_tags.append(ds_id)
            global_idx += 1

    assert len(embeddings) == 100

    # --------------------------------------------------------------
    # 5) pairwise RISWIE distances
    n_graphs = len(embeddings)
    dist_mat = np.zeros((n_graphs, n_graphs))

    print("Computing 100 × 100 RISWIE distances …")
    for i, (_, _, Ei) in enumerate(tqdm(embeddings)):
        for j in range(i+1, n_graphs):
            Ej = embeddings[j][2]
            #d = riswie_distance(Ei, Ej)
            #d = sliced_wasserstein_distance(Ei, Ej)
            d = gromov_wasserstein(Ei, Ej)
            dist_mat[i, j] = dist_mat[j, i] = d

    # --------------------------------------------------------------
    # 6) nearest-neighbour consistency score
    correct = 0
    for i in range(n_graphs):
        # indices sorted by distance (ascending), exclude self
        neighbours = np.argsort(dist_mat[i])[1:10]       # 9 nearest
        # how many share the same dataset id?
        same = sum(dataset_tags[j] == dataset_tags[i] for j in neighbours)
        correct += same

    accuracy = correct / (n_graphs * 9)
    print(f"\nNearest-neighbour consistency accuracy: {accuracy:.4f}")


# ---------------------------------------------------------------------
if __name__ == "__main__":
    main()


Computing 100 × 100 RISWIE distances …


  1%|          | 1/100 [42:17<69:46:55, 2537.53s/it]


KeyboardInterrupt: 